In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from Sinkhorn_CMI import SinkhornImputation_CMI
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
from CMI import *

from Inject_Missing_Values import *
from Experiment import *


import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [2]:
Visualize()

abc


In [3]:
d =  {
    "A": [10,10,20,40,70,40,50,80,10,40,20,60,50,30,20],
    "B": [1,1,2,4,7,4,5,8,1,4,2,6,5,3,2],
    "C":[15,15,30,60,105,60,75,120,15,60,30,90,75,45,30],
    "D":[45,30,60,32,58,63,25,55,46,78,92,60,34,27,35]

}
data = pd.DataFrame(d)
X= data.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = data.iloc[:, -1] 

In [4]:
print(X)

     A  B    C
0   10  1   15
1   10  1   15
2   20  2   30
3   40  4   60
4   70  7  105
5   40  4   60
6   50  5   75
7   80  8  120
8   10  1   15
9   40  4   60
10  20  2   30
11  60  6   90
12  50  5   75
13  30  3   45
14  20  2   30


In [7]:
miss_mcar, index_mcar = Generate_Missing_Value(X,Y,"MCAR",25,dependencies=None)

Total Missing Percentage MCAR: 24.44%
Coloumn Wise Missing Percentage:  A    46.666667
B    13.333333
C    13.333333
D     0.000000
dtype: float64


In [8]:


dependencies_MAR = {
    "A": {
        "influencers": ["C","B"],
        "condition": lambda row: row["C"]+row["B"] > 30  # Just a boolean condition
          # Separate probability function
    },
    "B": {
        "influencers": ["C"],
        "condition": lambda row: True,
        "probability": lambda row: 0.2 if row["C"] < 30 else 0.1
    }
}
miss_mar, index_mar = Generate_Missing_Value(X,Y,"MAR",10,dependencies_MAR)
print(miss_mar)

Total Missing Percentage MAR25: 6.67%
Coloumn Wise Missing Percentage:  A    20.000000
B     6.666667
C     0.000000
D     0.000000
dtype: float64
       A    B    C   D
0   10.0  NaN   15  45
1   10.0  1.0   15  30
2   20.0  2.0   30  60
3   40.0  4.0   60  32
4   70.0  7.0  105  58
5   40.0  4.0   60  63
6    NaN  5.0   75  25
7   80.0  8.0  120  55
8   10.0  1.0   15  46
9   40.0  4.0   60  78
10   NaN  2.0   30  92
11  60.0  6.0   90  60
12   NaN  5.0   75  34
13  30.0  3.0   45  27
14  20.0  2.0   30  35


In [16]:

dependencies_MNAR = {
    "A": {
        "condition": lambda row: True,
        "probability": lambda row: 0.3 if row["A"] < 30 else 0.1  # High earners are more likely to not report salary
    },
    "C": {
        "condition": lambda row: row["C"] > 50, # High BP patients may avoid reporting BP readings
        "probability": lambda row: 0.2 
    }
    
}
miss_mnar, index_mnar = Generate_Missing_Value(X,Y,"MNAR",10,dependencies_MNAR)
print(miss_mnar)

Total Missing Percentage MNAR: 6.67%
Coloumn Wise Missing Percentage:  A    20.000000
B     0.000000
C     6.666667
D     0.000000
dtype: float64
       A  B      C   D
0   10.0  1   15.0  45
1   10.0  1   15.0  30
2    NaN  2   30.0  60
3   40.0  4   60.0  32
4   70.0  7  105.0  58
5   40.0  4   60.0  63
6   50.0  5   75.0  25
7   80.0  8  120.0  55
8   10.0  1   15.0  46
9    NaN  4    NaN  78
10  20.0  2   30.0  92
11  60.0  6   90.0  60
12   NaN  5   75.0  34
13  30.0  3   45.0  27
14  20.0  2   30.0  35


In [47]:
import torch
import torch.nn as nn

class CMI:
    @staticmethod
    def soft_bucketize(data, bucket_specs):
        """
        Softly bucketizes numerical columns using differentiable binning.
        """
        bucketized_data = data.clone()  

        for col, bins in bucket_specs.items():
            feature = bucketized_data[:, col]  # Select column
            min_val, max_val = feature.min(), feature.max()
            bin_centers = torch.linspace(min_val, max_val, bins, device=data.device)

            # Compute soft bin assignments using Gaussian kernel
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.unsqueeze(0))
            sigma = (max_val - min_val) / bins  # Bandwidth
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()

            
            bucketized_data = torch.cat([
            bucketized_data[:, :col], 
            new_feature.unsqueeze(1),  
            bucketized_data[:, col+1:]
        ], dim=1)  

        return bucketized_data 
    @staticmethod
    def compute_probabilities_torch(data, bins=10):
        """Differentiable soft binning using Gaussian kernel smoothing."""
        min_val, max_val = data.min(), data.max()
        bin_centers = torch.linspace(min_val, max_val, bins, device=data.device)

        # Compute soft binning with Gaussian kernel
        distances = torch.abs(data.unsqueeze(1) - bin_centers.unsqueeze(0))
        sigma = (max_val - min_val) / bins
        soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
        soft_assignments /= soft_assignments.sum(dim=1, keepdim=True)

        return soft_assignments.mean(dim=0)  # Normalize

    @staticmethod
    def conditional_mutual_information(data, X_cols, Y_cols, Z_cols, bucket_specs):
        """
        Compute fully differentiable Conditional Mutual Information (CMI).
        """
        if not data.requires_grad:
            raise ValueError("Input data must have requires_grad=True.")

        # Bucketize data
        bucketized_data = CMI.soft_bucketize(data, bucket_specs)

        # Compute probability distributions
        prob_cache = {}
        columns_list = [Z_cols, X_cols + Z_cols, Y_cols + Z_cols, X_cols + Y_cols + Z_cols]
        keys = ['Z', 'XZ', 'YZ', 'XYZ']

        for key, cols in zip(keys, columns_list):
            prob_cache[key] = CMI.compute_probabilities_torch(bucketized_data, cols)

        # Retrieve precomputed probabilities
        P_Z = prob_cache['Z']
        P_XZ = prob_cache['XZ']
        P_YZ = prob_cache['YZ']
        P_XYZ = prob_cache['XYZ']

        print("P_XYZ shape:", P_XYZ.shape)
        print("P_XZ shape:", P_XZ.shape)
        print("P_YZ shape:", P_YZ.shape)
        print("P_Z shape:", P_Z.shape)

        # Ensure matching probability shapes
        min_size = min(P_XYZ.shape[1], P_XZ.shape[1], P_YZ.shape[1], P_Z.shape[1])
        P_XYZ, P_XZ, P_YZ, P_Z = P_XYZ[:, :min_size], P_XZ[:, :min_size], P_YZ[:, :min_size], P_Z[:, :min_size]

        # Compute safe log ratio for stability
        safe_ratio = ((P_XYZ * P_Z) / (P_XZ * P_YZ + 1e-10)).clamp(min=1.0)  # 🔥 Fix: Ensure ratio is ≥ 1

    
        cmi_values = P_XYZ * torch.log2(safe_ratio)
        cmi = torch.sum(cmi_values)

    
        cmi = torch.clamp(cmi, min=0.0)

        return cmi


In [48]:
import pandas as pd

# Read CSV file into a Pandas DataFrame
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\Fair_Imputation\Data\Diabetic_Ground_Truth.csv', delimiter=',', header=0)

# Store feature columns in a DataFrame


X= groundTruth.iloc[:, :-1]  # Selects all rows and all columns except the last one
Y = groundTruth.iloc[:, -1]  # Selects all rows and only the last column


In [49]:
groundTruth_tensor = torch.tensor(scale(groundTruth)) #converting groundTruth to Tensor, z-score scaling
groundTruth_tensor.requires_grad = True

In [50]:
bucket_specs = {
     
    0: 4,   # Column 0 → age (4 bins)
    1: 2,  # Column 1 → gender (2 bins)
    17: 2,  # Column 17 → label  (2 bins)
       # Column 2 → Family Diabetic (2 bins)
       # Column 3 → HighBP (2 bins)
    4: 4, #column 4 ->  PhysicallyActive (4 bins)
    5: 20 #column 5-> BMI (20 bins)
    
}

# Define multiple attributes for X, Y, Z
X_cols = [0,1]  # Bucketized sensitive attributes (e.g., sex, race, age)
Y_cols = [17]     # Bucketized outcome-related attributes
Z_cols = [3,2,4,5,6,7]  


In [51]:
d= CMI.soft_bucketize(groundTruth_tensor,bucket_specs)

In [52]:
print(torch.unique(d[:,15]))

tensor([-0.1254,  7.9776], grad_fn=<Unique2Backward0>)


In [53]:

# Compute Conditional Mutual Information (CMI)
cmi_value = CMI.conditional_mutual_information(groundTruth_tensor, X_cols, Y_cols, Z_cols, bucket_specs)


print("CMI Estimate:", cmi_value)
print("CMI grad_fn:", cmi_value.grad_fn)  # Should NOT be None


cmi_value.backward()


#print("Gradient of data:", data.grad)  # Should NOT be None


TypeError: linspace() received an invalid combination of arguments - got (Tensor, Tensor, list, device=torch.device), but expected one of:
 * (Tensor start, Tensor end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Tensor end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Tensor start, Number end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [54]:
import numpy as np
from sklearn.neighbors import KernelDensity

def estimate_entropy(kde, samples):
    """Estimate differential entropy using KDE"""
    log_prob = kde.score_samples(samples)
    return -np.mean(log_prob)

def conditional_mutual_information(X, Y, Z, bandwidth=0.2):
    """Estimate Conditional Mutual Information I(X;Y | Z) using KDE"""
    
    # Stack variables for joint distributions
    XZ = np.vstack([X, Z]).T
    YZ = np.vstack([Y, Z]).T
    XYZ = np.vstack([X, Y, Z]).T
    Z_reshaped = Z.reshape(-1, 1)
    
    # Fit KDE models
    kde_xz = KernelDensity(bandwidth=bandwidth).fit(XZ)
    kde_yz = KernelDensity(bandwidth=bandwidth).fit(YZ)
    kde_xyz = KernelDensity(bandwidth=bandwidth).fit(XYZ)
    kde_z = KernelDensity(bandwidth=bandwidth).fit(Z_reshaped)

    # Compute entropies
    h_x_given_z = estimate_entropy(kde_xz, XZ) - estimate_entropy(kde_z, Z_reshaped)
    h_y_given_z = estimate_entropy(kde_yz, YZ) - estimate_entropy(kde_z, Z_reshaped)
    h_xy_given_z = estimate_entropy(kde_xyz, XYZ) - estimate_entropy(kde_z, Z_reshaped)

    # Compute Conditional Mutual Information
    cmi = h_x_given_z + h_y_given_z - h_xy_given_z
    return cmi

# Generate synthetic data
np.random.seed(42)
Z = np.random.uniform(-1, 1, 1000)  # Conditioning variable
X = Z + np.random.normal(0, 0.5, 1000)  # X depends on Z
Y = Z + np.random.normal(0, 0.5, 1000)  # Y depends on Z

# Compute Conditional Mutual Information
cmi_value = conditional_mutual_information(X, Y, Z)
print(f"Estimated Conditional Mutual Information I(X;Y | Z): {cmi_value:.4f}")


Estimated Conditional Mutual Information I(X;Y | Z): 0.0542


In [55]:
import torch

def gaussian_kernel(x, y, sigma=0.2):
    """Computes Gaussian kernel similarity"""
    diff = x.unsqueeze(1) - y.unsqueeze(0)
    return torch.exp(-torch.sum(diff**2, dim=-1) / (2 * sigma**2))

def kde_entropy(samples, sigma=0.2):
    """Differentiable KDE-based entropy estimation"""
    n = samples.shape[0]
    kernel_matrix = gaussian_kernel(samples, samples, sigma)
    density_estimates = kernel_matrix.mean(dim=1)
    entropy = -torch.mean(torch.log(density_estimates + 1e-10))  # Prevent log(0)
    return entropy

def conditional_mutual_information_kde(X, Y, Z, sigma=0.2):
    """Computes I(X; Y | Z) using differentiable KDE"""
    h_x_given_z = kde_entropy(torch.cat([X, Z], dim=1), sigma) - kde_entropy(Z, sigma)
    h_y_given_z = kde_entropy(torch.cat([Y, Z], dim=1), sigma) - kde_entropy(Z, sigma)
    h_xy_given_z = kde_entropy(torch.cat([X, Y, Z], dim=1), sigma) - kde_entropy(Z, sigma)
    return h_x_given_z + h_y_given_z - h_xy_given_z

# Generate synthetic data (same as before)
N = 1000
Z = torch.randn(N, 1, requires_grad=True)  # Conditioning variable
X = Z + 0.5 * torch.randn(N, 1, requires_grad=True)  # X depends on Z
Y = Z + 0.5 * torch.randn(N, 1, requires_grad=True)  # Y depends on Z

# Compute CMI with differentiable KDE
cmi_value = conditional_mutual_information_kde(X, Y, Z)
print(f"Estimated Conditional Mutual Information I(X; Y | Z): {cmi_value.item():.4f}")

# Compute gradient with respect to Z
cmi_value.backward()
print(f"Gradient of CMI wrt Z: {Z.grad[:5].flatten()}")  # Check gradient values


Estimated Conditional Mutual Information I(X; Y | Z): 0.0997
Gradient of CMI wrt Z: tensor([ 0.0020,  0.0014, -0.0006, -0.0027, -0.0026])


In [57]:
import torch

def gaussian_kernel(x, y, sigma=0.2):
    """Computes Gaussian kernel similarity"""
    diff = x.unsqueeze(1) - y.unsqueeze(0)  # Pairwise differences
    return torch.exp(-torch.sum(diff**2, dim=-1) / (2 * sigma**2))  # Gaussian Kernel

def kde_entropy(samples, sigma=0.2):
    """Differentiable Kernel Density Estimation (KDE) for entropy estimation"""
    kernel_matrix = gaussian_kernel(samples, samples, sigma)
    density_estimates = kernel_matrix.mean(dim=1)
    entropy = -torch.mean(torch.log(density_estimates + 1e-10))  # Prevent log(0)
    return entropy

def conditional_differential_entropy(X, Z, sigma=0.2):
    """Computes conditional differential entropy h(X|Z) = h(X,Z) - h(Z)"""
    XZ = torch.cat([X, Z], dim=1)  # Concatenating X and Z
    h_xz = kde_entropy(XZ, sigma)
    h_z = kde_entropy(Z, sigma)
    return h_xz - h_z  # Conditional entropy h(X|Z)

def conditional_mutual_information(X, Y, Z, sigma=0.2):
    """Computes differentiable Conditional Mutual Information I(X; Y | Z)"""
    h_x_given_z = conditional_differential_entropy(X, Z, sigma)
    h_y_given_z = conditional_differential_entropy(Y, Z, sigma)
    h_xy_given_z = conditional_differential_entropy(torch.cat([X, Y], dim=1), Z, sigma)

    # Compute CMI
    cmi = h_x_given_z + h_y_given_z - h_xy_given_z
    return cmi

# Generate Multi-Dimensional Continuous Data (With AutoGrad Enabled)
torch.manual_seed(42)
N = 1000  # Number of samples
dim_X, dim_Y, dim_Z = 2, 2, 2  # Dimensions

Z = torch.randn(N, dim_Z, requires_grad=True)  # Conditioning variables
X = Z @ torch.randn(dim_Z, dim_X) + 0.5 * torch.randn(N, dim_X, requires_grad=True)  # X depends on Z
Y = Z @ torch.randn(dim_Z, dim_Y) + 0.5 * torch.randn(N, dim_Y, requires_grad=True)  # Y depends on Z

# Compute Differentiable Conditional Mutual Information
cmi_value = conditional_mutual_information(X, Y, Z)
print(f"Estimated Conditional Mutual Information: {cmi_value.item():.4f}")

# Compute Gradient with respect to Z
cmi_value.backward()
print(f"Gradient of CMI wrt Z: {Z.grad[:5].flatten()}")  # Displaying first few gradients


Estimated Conditional Mutual Information: 1.4053
Gradient of CMI wrt Z: tensor([ 0.0043,  0.0036,  0.0011, -0.0012, -0.0028,  0.0042, -0.0004,  0.0012,
        -0.0077,  0.0005])


In [59]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

# Simulated real-world dataset with multiple X, Y, Z features
N = 1000  # Number of samples
dim_X, dim_Y, dim_Z = 2, 1, 3  # Multi-dimensional features

# Generate a synthetic dataset (replace with real-world data)
torch.manual_seed(42)
Z = torch.randn(N, dim_Z, requires_grad=True)  # Conditioning variables (3D)
X = Z @ torch.randn(dim_Z, dim_X) + 0.5 * torch.randn(N, dim_X, requires_grad=True)  # X depends on Z
Y = Z @ torch.randn(dim_Z, dim_Y) + 0.5 * torch.randn(N, dim_Y, requires_grad=True)  # Y depends on Z

def gaussian_kernel(x, y, sigma=0.2):
    """Computes Gaussian kernel similarity"""
    diff = x.unsqueeze(1) - y.unsqueeze(0)  # Pairwise differences
    return torch.exp(-torch.sum(diff**2, dim=-1) / (2 * sigma**2))  # Gaussian Kernel

def kde_entropy(samples, sigma=0.2):
    """Differentiable KDE-based entropy estimation"""
    kernel_matrix = gaussian_kernel(samples, samples, sigma)
    density_estimates = kernel_matrix.mean(dim=1)
    entropy = -torch.mean(torch.log(density_estimates + 1e-10))  # Prevent log(0)
    return entropy

def conditional_differential_entropy(X, Z, sigma=0.2):
    """Computes conditional differential entropy h(X|Z) = h(X,Z) - h(Z)"""
    XZ = torch.cat([X, Z], dim=1)  # Concatenating X and Z
    h_xz = kde_entropy(XZ, sigma)
    h_z = kde_entropy(Z, sigma)
    return h_xz - h_z  # Conditional entropy h(X|Z)

def conditional_mutual_information(X, Y, Z, sigma=0.2):
    """Computes differentiable Conditional Mutual Information I(X; Y | Z)"""
    h_x_given_z = conditional_differential_entropy(X, Z, sigma)
    h_y_given_z = conditional_differential_entropy(Y, Z, sigma)
    h_xy_given_z = conditional_differential_entropy(torch.cat([X, Y], dim=1), Z, sigma)

    # Compute CMI
    cmi = h_x_given_z + h_y_given_z - h_xy_given_z
    return cmi

# Compute CMI on Multi-Dimensional Data
cmi_value = conditional_mutual_information(X, Y, Z)

# Compute Gradient with respect to Z
cmi_value.backward()

# Display results
{
    "Estimated Conditional Mutual Information": cmi_value.item(),
    "Gradient of CMI wrt Z (First 5 Values)": Z.grad[:5].flatten().tolist()
}


{'Estimated Conditional Mutual Information': 0.5821194434625134,
 'Gradient of CMI wrt Z (First 5 Values)': [-0.0017520666312357226,
  0.0023430754199654973,
  -0.0036632118552742997,
  -0.00024082595278532992,
  0.002346146218324777,
  0.0020583589071262974,
  0.0014867170125502354,
  0.00039828858704201467,
  -0.000707830809133528,
  0.0020481994992923667,
  0.001364668192037918,
  0.0017069824657126162,
  0.00229573262244321,
  0.0004573957341224032,
  0.0066925142567881265]}

In [81]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

def kde_entropy_torch(samples, sigma=0.2):
    """Estimate differential entropy using a Gaussian Kernel with PyTorch (AutoGrad enabled)."""
    pairwise_distances = torch.cdist(samples, samples, p=2)  # Euclidean distance
    kernel_matrix = torch.exp(-pairwise_distances ** 2 / (2 * sigma ** 2))
    density_estimates = kernel_matrix.mean(dim=1)
    entropy = -torch.mean(torch.log(density_estimates + 1e-10))  # Prevent log(0)
    return entropy

def conditional_differential_entropy_torch(X, Z, sigma=0.2):
    """Computes conditional differential entropy h(X|Z) = h(X,Z) - h(Z) with AutoGrad support"""
    XZ = torch.cat([X, Z], dim=1)  # Concatenating X and Z
    h_xz = kde_entropy_torch(XZ, sigma)
    h_z = kde_entropy_torch(Z, sigma)
    return h_xz - h_z  # Conditional entropy h(X|Z)

def conditional_mutual_information_torch(X, Y, Z, sigma=0.2):
    """Computes differentiable Conditional Mutual Information I(X; Y | Z)"""
    h_x_given_z = conditional_differential_entropy_torch(X, Z, sigma)
    h_y_given_z = conditional_differential_entropy_torch(Y, Z, sigma)
    h_xy_given_z = conditional_differential_entropy_torch(torch.cat([X, Y], dim=1), Z, sigma)

    # Compute CMI
    cmi = h_x_given_z + h_y_given_z - h_xy_given_z
    return cmi

def compute_cmi_from_csv_with_grad(file_path, x_cols, y_cols, z_cols):
    """Loads dataset, selects columns, normalizes, and computes differentiable CMI"""
    # Load dataset
    df = pd.read_csv(file_path)

    # Select columns
    X = df.iloc[:, x_cols].values  # Select X columns
    Y = df.iloc[:, y_cols].values  # Select Y columns
    Z = df.iloc[:, z_cols].values  # Select Z columns

    # Normalize data for stability in entropy estimation
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    Y = scaler.fit_transform(Y)
    Z = scaler.fit_transform(Z)

    # Convert to PyTorch tensors with requires_grad=True for AutoGrad
    X_torch = torch.tensor(X, dtype=torch.float32, requires_grad=True)
    Y_torch = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
    Z_torch = torch.tensor(Z, dtype=torch.float32, requires_grad=True)

    # Compute CMI (Differentiable)
    cmi_value = conditional_mutual_information_torch(X_torch, Y_torch, Z_torch)
    print(f"CMI Tensor: {cmi_value}, grad_fn: {cmi_value.grad_fn}")
    
    # Compute Gradient with respect to Z
    cmi_value.backward()

    return cmi_value, Z_torch.grad

# Example Usage
file_path = (r'C:\Users\zhossai3\Desktop\Fair_Imputation\Data\Diabetic_Ground_Truth.csv') # Replace with your actual CSV file
x_cols = [ 0,1]  # Column indices for X
y_cols = [17]  # Column indices for Y
z_cols = [4,5]  # Column indices for Z

# Compute Conditional Mutual Information
cmi_result, z_grad = compute_cmi_from_csv_with_grad(file_path, x_cols, y_cols, z_cols)
print(f"Estimated Conditional Mutual Information: {cmi_result}")
print(f"Gradient of CMI with respect to Z (first 5 values): {z_grad[:5].flatten().tolist()}")



CMI Tensor: 0.3615305423736572, grad_fn: <SubBackward0 object at 0x000001F8A6519030>
Estimated Conditional Mutual Information: 0.3615305423736572
Gradient of CMI with respect to Z (first 5 values): [-1.862645149230957e-09, 4.6566128730773926e-07, -4.0978193283081055e-08, -0.000372511800378561, 2.7348287403583527e-06, 0.001090371049940586, -1.2484160833992064e-06, -0.0027197087183594704, -1.4062970876693726e-06, 0.0008600093424320221]


In [7]:
groundTruth = pd.read_csv(r'C:\Users\zhossai3\Desktop\Fair_Imputation\Data\Diabetic_Ground_Truth.csv', delimiter=',', header=0)
groundTruth_tensor = torch.tensor(scale(groundTruth))

In [35]:
import torch
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

def bucketize_tensor(data, bucket_specs):
    """Bucketizes each column of the PyTorch tensor based on `bucket_specs`."""
    bucketized_data = data.clone()  

    for col, bins in bucket_specs.items():
            feature = bucketized_data[:, col]  # Select column
            min_val, max_val = feature.min(), feature.max()
            bin_centers = torch.linspace(min_val, max_val, bins, device=data.device)

            # Compute soft bin assignments using Gaussian kernel
            distances = torch.abs(feature.unsqueeze(1) - bin_centers.unsqueeze(0))
            sigma = (max_val - min_val) / bins  # Bandwidth
            soft_assignments = torch.exp(-0.5 * (distances / sigma) ** 2)
            soft_assignments = soft_assignments / soft_assignments.sum(dim=1, keepdim=True)

            # Compute differentiable bin values
            new_feature = (soft_assignments @ bin_centers).squeeze()

            
            bucketized_data = torch.cat([
            bucketized_data[:, :col], 
            new_feature.unsqueeze(1),  
            bucketized_data[:, col+1:]
        ], dim=1)  

    return bucketized_data  # Convert back to tensor

def kde_entropy_torch(samples, sigma=0.2):
    """Estimate differential entropy using a Gaussian Kernel with PyTorch (AutoGrad enabled)."""
    pairwise_distances = torch.cdist(samples, samples, p=2)  # Euclidean distance
    kernel_matrix = torch.exp(-pairwise_distances ** 2 / (2 * sigma ** 2))
    density_estimates = kernel_matrix.mean(dim=1)
    entropy = -torch.mean(torch.log(density_estimates + 1e-10))  # Prevent log(0)
    return entropy

def conditional_differential_entropy_torch(X, Z, sigma=0.2):
    """Computes conditional differential entropy h(X|Z) = h(X,Z) - h(Z)"""
    XZ = torch.cat([X, Z], dim=1)  # Concatenating X and Z
    h_xz = kde_entropy_torch(XZ, sigma)
    h_z = kde_entropy_torch(Z, sigma)
    return h_xz - h_z  # Conditional entropy h(X|Z)

def conditional_mutual_information_torch(X, Y, Z, sigma=0.2):
    """Computes differentiable Conditional Mutual Information I(X; Y | Z)"""
    h_x_given_z = conditional_differential_entropy_torch(X, Z, sigma)
    h_y_given_z = conditional_differential_entropy_torch(Y, Z, sigma)
    h_xy_given_z = conditional_differential_entropy_torch(torch.cat([X, Y], dim=1), Z, sigma)

    # Compute CMI
    cmi = h_x_given_z + h_y_given_z - h_xy_given_z
    return cmi

def compute_cmi_with_bucketized_tensor(data_tensor, bucket_specs, X_cols, Y_cols, Z_cols):
    """
    Applies bucketization, selects columns, and computes differentiable CMI.
    
    Args:
    - data_tensor: Input PyTorch tensor.
    - bucket_specs: Dictionary specifying number of bins per column.
    - X_cols, Y_cols, Z_cols: Lists of column indices for X, Y, Z.
    
    Returns:
    - cmi_value: Estimated CMI.
    - z_grad: Gradient of CMI w.r.t. Z.
    """
    # Apply bucketization
    bucketized_tensor = bucketize_tensor(data_tensor, bucket_specs)

    # Select features for X, Y, Z
    X_torch = bucketized_tensor[:, X_cols].clone().detach().requires_grad_(True)
    Y_torch = bucketized_tensor[:, Y_cols].clone().detach().requires_grad_(True)
    Z_torch = bucketized_tensor[:, Z_cols].clone().detach().requires_grad_(True)

    # Compute CMI (Differentiable)
    cmi_value = conditional_mutual_information_torch(X_torch, Y_torch, Z_torch)

    # Compute Gradient with respect to Z
   

    return cmi_value

# Example Input Data (Replace with your real tensor)
torch.manual_seed(42)
  # Example tensor with 8 features

# Bucketization Dictionary
bucket_specs = {
    0: 4,   # Column 0 → age (4 bins)
    1: 2,   # Column 1 → gender (2 bins)
    17: 2,  # Column 17 → label (2 bins)
    4: 4,   # Column 4 → Physically Active (4 bins)
    5: 20   # Column 5 → BMI (20 bins)
}

# Column Selection
X_cols = [0, 1]         # Sensitive attributes
Y_cols = [17]           # Outcome-related attributes
Z_cols = [4,5,15]  # Additional covariates

# Compute CMI with Bucketization
cmi_result = compute_cmi_with_bucketized_tensor(groundTruth_tensor, bucket_specs, X_cols, Y_cols, Z_cols)

print(f"Estimated Conditional Mutual Information (Bucketized Data): {cmi_result:.4f}")
#print(f"Gradient of CMI with respect to Z (first 5 values): {z_grad[:5].flatten().tolist()}")


Estimated Conditional Mutual Information (Bucketized Data): 0.3567
